In [ ]:
import torch
import numpy as np
import pandas as pd

from models.resnet_gkt import ResNet49, ResNet8
from utils.datasets import get_datasets
from utils.sampling import get_user_groups
from fedgkt.trainers import GKTServerTrainer, GKTClientTrainer

In [ ]:
device = 'cuda' if torch.cuda.is_available else 'cpu'

In [ ]:
seed = 0  # 128, 479

In [ ]:
# setting parameters
client_number = 100
norm_type = 'Group Norm'
participation_frac = 0.1
iid = True
unbalanced = False
ROUNDS = 50

args_server = {
    'temperature': 3,
    'epochs_server': 10,
    'alpha': 0.5
}

args_client={
    'temperature': 3,
    'epochs_client': 1,
    'alpha': 0.5
}

In [ ]:
metric = []

df_train = pd.DataFrame()
df_test = pd.DataFrame()

for iid, unbalanced, norm in [
        [True, False, "Batch Norm"], [False, True, "Batch Norm"], [False, False, "Batch Norm"],
        [True, False, "Group Norm"], [False, True, "Group Norm"], [False, False, "Group Norm"]]:
    # server, reproducibility demanded to the server
    server_model = ResNet49(norm_type)
    server_trainer = GKTServerTrainer(client_number, device, server_model, args_server, seed=seed)

    # client
    client_model = ResNet8(norm_type)
    trainset, testset = get_datasets(augmentation=True)
    user_groups = get_user_groups(trainset, iid=iid, unbalanced=unbalanced, tot_users=client_number)

    clients = []
    for client_idx in range(client_number):
        clients.append(GKTClientTrainer(client_idx, trainset, testset,
                                        user_groups[client_idx], device, client_model, args_client))

    for round in range(ROUNDS):
        print("Communication round: ", round+1)
        m = max(int(participation_frac*client_number), 1)
        chosen_users = np.random.choice(range(client_number), m, replace=False)
        print(f"Chosen users: {chosen_users}")
        for idx in chosen_users:
            extracted_features_dict, extracted_logits_dict, labels_dict,\
            extracted_features_dict_test, labels_dict_test = clients[idx].train()

            server_trainer.add_local_trained_result(idx, extracted_features_dict, extracted_logits_dict, labels_dict,\
            extracted_features_dict_test, labels_dict_test)

        server_trainer.train(round)

        for idx in chosen_users:
            global_logits = server_trainer.get_global_logits(idx)
            clients[idx].update_large_model_logits(global_logits)
        server_trainer.remove_records()

    train_metrics, test_metrics = server_trainer.get_metrics_lists()
    
    train_data = pd.DataFrame(train_metrics)
    train_data["norm"] = "BN" if norm == "Batch Norm" else "GN"
    train_data["independence"] = "iid" if iid else "noniid"
    train_data["balancement"] = "unbalanced" if unbalanced else "balanced"
    train_data["seed"] = seed

    test_data = pd.DataFrame(test_metrics)
    test_data["norm"] = "BN" if norm == "Batch Norm" else "GN"
    test_data["independence"] = "iid" if iid else "noniid"
    test_data["balancement"] = "unbalanced" if unbalanced else "balanced"
    test_data["seed"] = seed

    df_train = pd.concat([df_train, train_data], ignore_index=True)
    df_test = pd.concat([df_test, test_data], ignore_index=True)

df_train.to_csv("./results/federated_gkt/fedgkt_train_results.csv", index=False)
df_test.to_csv("./results/federated_gkt/fedgkt_test_results.csv", index=False)